<a href="https://colab.research.google.com/github/MinkyuRamen/KubigCVPoseEstiProject/blob/main/classification_jiyoon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 임포트 및 드라이브 마운트

In [ ]:
!git clone https://github.com/JunkyByte/easy_ViTPose.git
%cd easy_ViTPose
!pip install -r requirements.txt
!pip install -e .
!pip install huggingface_hub
!pip install onnx onnxruntime-gpu

Cloning into 'easy_ViTPose'...
remote: Enumerating objects: 682, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 682 (delta 55), reused 62 (delta 26), pack-reused 583
Receiving objects: 100% (682/682), 8.71 MiB | 27.62 MiB/s, done.
Resolving deltas: 100% (392/392), done.
/content/easy_ViTPose
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 610.8/610.8 kB 27.1

Obtaining file:///content/easy_ViTPose
  Preparing metadata (setup.py) ... done
  Running setup.py develop for easy-ViTPose
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 37.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import cv2
import os
from google.colab import files

from huggingface_hub import hf_hub_download

import numpy as np
from PIL import Image
from urllib.request import urlopen
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

# ViTPose main_code edit

In [ ]:
MODEL_TYPE = 'onnx'
MODEL_SIZE = 'l'
YOLO_TYPE = 'onnx'
YOLO_SIZE = 'n'
ext = {'tensorrt': '.engine', 'onnx': '.onnx', 'torch': '.pth'}[MODEL_TYPE]
ext_yolo = {'onnx': '.onnx', 'torch': '.pt'}[YOLO_TYPE]

REPO_ID = 'JunkyByte/easy_ViTPose'
FILENAME = os.path.join(MODEL_TYPE, 'vitpose-25-' + MODEL_SIZE) + ext
FILENAME_YOLO = 'yolov5/yolov5' + YOLO_SIZE + ext_yolo

print(f'Downloading model {REPO_ID}/{FILENAME}')
model_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
yolo_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME_YOLO)

In [ ]:
%cd /content/easy_ViTPose/easy_ViTPose/
!ls

/content/easy_ViTPose/easy_ViTPose
configs      easy_ViTPose.egg-info  sort.py	   train.py	    vit_utils
config.yaml  inference.py	    to_onnx.ipynb  utils_engine.py
datasets     __init__.py	    to_trt.ipynb   vit_models


In [ ]:
# If you get an error from PIL restart environment and rerun this cell to update packages version
# from easy_ViTPose.inference import VitInference
import abc
import time
from typing import Optional
import json
import os
import tqdm

from PIL import Image
import cv2
import numpy as np
import torch

from configs.ViTPose_common import data_cfg
from vit_models.model import ViTPose
from vit_utils.top_down_eval import keypoints_from_heatmaps
from vit_utils.visualization import draw_points_and_skeleton, joints_dict
from vit_utils.inference import pad_image, VideoReader, NumpyEncoder, draw_bboxes
from sort import Sort

try:  # Add bools -> error stack
    import pycuda.driver as cuda  # noqa: [F401]
    import pycuda.autoinit  # noqa: [F401]
    import utils_engine as engine_utils
    import tensorrt as trt
    has_trt = True
except ModuleNotFoundError:
    pass

try:
    import onnxruntime
    has_onnx = True
except ModuleNotFoundError:
    pass

__all__ = ['VitInference']
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]


class VitInference:
    """
    Class for performing inference using ViTPose models with YOLOv5 human detection detection and SORT tracking.

    Args:
        model (str): Path to the ViT model file (.pth, .onnx, .engine).
        yolo_name (str): Name of the YOLOv5 model to load.
        model_name (str, optional): Name of the ViT model architecture to use. Valid values are 's', 'b', 'l', 'h'.
                                    Defaults to None, is necessary when using .pth checkpoints.
        yolo_size (int, optional): Size of the input image for YOLOv5 model. Defaults to 320.
        device (str, optional): Device to use for inference. Defaults to 'cuda' if available, else 'cpu'.
        is_video (bool, optional): Flag indicating if the input is video. Defaults to False.
        single_pose (bool, optional): Flag indicating if the video (on images this flag has no effect) will contain a single pose.
                                      In this case the SORT tracker is not used (increasing performance) but people id tracking
                                      won't be consistent among frames.
        yolo_step (int, optional): The tracker can be used to predict the bboxes instead of yolo for performance,
                                   this flag specifies how often yolo is applied (e.g. 1 applies yolo every frame).
                                   This does not have any effect when is_video is False.
    """

    def __init__(self, model: str,
                 yolo_name: str,
                 model_name: Optional[str] = None,
                 yolo_size: Optional[int] = 320,
                 device: Optional[str] = None,
                 is_video: Optional[bool] = False,
                 single_pose: Optional[bool] = False,
                 yolo_step: Optional[int] = 1):
        assert os.path.isfile(model), f'The model file {model} does not exist'
        assert os.path.isfile(yolo_name), f'The YOLOv5 model {yolo_name} does not exist'

        # Device priority is cuda / mps / cpu
        if device is None:
            if torch.cuda.is_available():
                device = 'cuda'
            elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
                device = 'mps'
            else:
                device = 'cpu'

        self.device = torch.device(device)
        self.yolo = torch.hub.load("ultralytics/yolov5", "custom", yolo_name)
        self.yolo.to(self.device)
        self.yolo.classes = [0]
        self.yolo_size = yolo_size
        self.yolo_step = yolo_step
        self.is_video = is_video
        self.single_pose = single_pose
        self.reset()

        # State saving during inference
        self.save_state = True  # Can be disabled manually
        self._img = None
        self._yolo_res = None
        self._tracker_res = None
        self._keypoints = None

        # Use extension to decide which kind of model has been loaded
        use_onnx = model.endswith('.onnx')
        use_trt = model.endswith('.engine')

        assert model_name in [None, 's', 'b', 'l', 'h'], \
            f'The model name {model_name} is not valid'

        # onnx / trt models do not require model_cfg specification
        if model_name is None:
            assert use_onnx or use_trt, \
                'Specify the model_name if not using onnx / trt'
        else:
            if model_name == 's':
                from configs.ViTPose_small_coco_256x192 import model as model_cfg
            elif model_name == 'b':
                from configs.ViTPose_base_coco_256x192 import model as model_cfg
            elif model_name == 'l':
                from configs.ViTPose_large_coco_256x192 import model as model_cfg
            elif model_name == 'h':
                from configs.ViTPose_huge_coco_256x192 import model as model_cfg

        self.target_size = data_cfg['image_size']
        if use_onnx:
            self._ort_session = onnxruntime.InferenceSession(model,
                                                             providers=['CUDAExecutionProvider',
                                                                        'CPUExecutionProvider'])
            inf_fn = self._inference_onnx
        elif use_trt:
            logger = trt.Logger(trt.Logger.ERROR)
            trt_runtime = trt.Runtime(logger)
            trt_engine = engine_utils.load_engine(trt_runtime, model)

            # This allocates memory for network inputs/outputs on both CPU and GPU
            self._inputs, self._outputs, self._bindings, self._stream = \
                engine_utils.allocate_buffers(trt_engine)
            # Execution context is needed for inference
            self._context = trt_engine.create_execution_context()
            inf_fn = self._inference_trt
        else:
            self._vit_pose = ViTPose(model_cfg)
            self._vit_pose.eval()

            ckpt = torch.load(model, map_location='cpu')
            if 'state_dict' in ckpt:
                self._vit_pose.load_state_dict(ckpt['state_dict'])
            else:
                self._vit_pose.load_state_dict(ckpt)
            self._vit_pose.to(device)
            inf_fn = self._inference_torch

        # Override inference with selected engine
        self._inference = inf_fn

    def reset(self):
        """
        Reset the inference class to be ready for a new video.
        This will reset the internal counter of frames, on videos
        this is necessary to reset the tracker.
        """
        min_hits = 3 if self.yolo_step == 1 else 1
        use_tracker = self.is_video and not self.single_pose
        self.tracker = Sort(max_age=self.yolo_step,
                            min_hits=min_hits,
                            iou_threshold=0.3) if use_tracker else None  # TODO: Params
        self.frame_counter = 0

    @classmethod
    def postprocess(cls, heatmaps, org_w, org_h):
        """
        Postprocess the heatmaps to obtain keypoints and their probabilities.

        Args:
            heatmaps (ndarray): Heatmap predictions from the model.
            org_w (int): Original width of the image.
            org_h (int): Original height of the image.

        Returns:
            ndarray: Processed keypoints with probabilities.
        """
        points, prob = keypoints_from_heatmaps(heatmaps=heatmaps,
                                               center=np.array([[org_w // 2,
                                                                 org_h // 2]]),
                                               scale=np.array([[org_w, org_h]]),
                                               unbiased=True, use_udp=True)
        return np.concatenate([points[:, :, ::-1], prob], axis=2)

    @abc.abstractmethod
    def _inference(img: np.ndarray) -> np.ndarray:
        """
        Abstract method for performing inference on an image.
        It is overloaded by each inference engine.

        Args:
            img (ndarray): Input image for inference.

        Returns:
            ndarray: Inference results.
        """
        raise NotImplementedError

    def inference(self, img: np.ndarray) -> np.ndarray:
        """
        Perform inference on the input image.

        Args:
            img (ndarray): Input image for inference in RGB format.

        Returns:
            ndarray: Inference results.
        """

        # First use YOLOv5 for detection
        res_pd = np.empty((0, 5))
        results = None
        if (self.tracker is None or
           (self.frame_counter % self.yolo_step == 0 or self.frame_counter < 3)):
            results = self.yolo(img, size=self.yolo_size)
            res_pd = np.array([r[:5].tolist() for r in  # TODO: Confidence threshold
                               results.pandas().xyxy[0].to_numpy() if r[4] > 0.35]).reshape((-1, 5))
        self.frame_counter += 1

        frame_keypoints = {}
        ids = None
        if self.tracker is not None:
            res_pd = self.tracker.update(res_pd)
            ids = res_pd[:, 5].astype(int).tolist()

        # Prepare boxes for inference
        bboxes = res_pd[:, :4].round().astype(int)
        scores = res_pd[:, 4].tolist()
        pad_bbox = 10

        if ids is None:
            ids = range(len(bboxes))

        for bbox, id in zip(bboxes, ids):
            # TODO: Slightly bigger bbox
            bbox[[0, 2]] = np.clip(bbox[[0, 2]] + [-pad_bbox, pad_bbox], 0, img.shape[1])
            bbox[[1, 3]] = np.clip(bbox[[1, 3]] + [-pad_bbox, pad_bbox], 0, img.shape[0])

            # Crop image and pad to 3/4 aspect ratio
            img_inf = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
            img_inf, (left_pad, top_pad) = pad_image(img_inf, 3 / 4)

            keypoints = self._inference(img_inf)[0]
            # Transform keypoints to original image
            keypoints[:, :2] += bbox[:2][::-1] - [top_pad, left_pad]
            frame_keypoints[id] = keypoints

        if self.save_state:
            self._img = img
            self._yolo_res = results
            self._tracker_res = (bboxes, ids, scores)
            self._keypoints = frame_keypoints

        return frame_keypoints

    def draw(self, show_yolo=True, show_raw_yolo=False, confidence_threshold=0.5):
        """
        Draw keypoints and bounding boxes on the image.

        Args:
            show_yolo (bool, optional): Whether to show YOLOv5 bounding boxes. Default is True.
            show_raw_yolo (bool, optional): Whether to show raw YOLOv5 bounding boxes. Default is False.

        Returns:
            ndarray: Image with keypoints and bounding boxes drawn.
        """
        img = self._img.copy()
        bboxes, ids, scores = self._tracker_res

        if self._yolo_res is not None and (show_raw_yolo or (self.tracker is None and show_yolo)):
            img = np.array(self._yolo_res.render())[0]

        if show_yolo and self.tracker is not None:
            img = draw_bboxes(img, bboxes, ids, scores)

        img = np.array(img)[..., ::-1]  # RGB to BGR for cv2 modules
        for idx, k in self._keypoints.items():
            img = draw_points_and_skeleton(img.copy(), k,
                                           joints_dict()['coco']['skeleton'],
                                           person_index=idx,
                                           points_color_palette='gist_rainbow',
                                           skeleton_color_palette='jet',
                                           points_palette_samples=10,
                                           confidence_threshold=confidence_threshold)
        return img[..., ::-1], self._keypoints  # Return RGB as original

    def pre_img(self, img):
        org_h, org_w = img.shape[:2]
        img_input = cv2.resize(img, self.target_size, interpolation=cv2.INTER_LINEAR) / 255
        img_input = ((img_input - MEAN) / STD).transpose(2, 0, 1)[None].astype(np.float32)
        return img_input, org_h, org_w

    @torch.no_grad()
    def _inference_torch(self, img: np.ndarray) -> np.ndarray:
        # Prepare input data
        img_input, org_h, org_w = self.pre_img(img)
        img_input = torch.from_numpy(img_input).to(self.device)

        # Feed to model
        heatmaps = self._vit_pose(img_input).detach().cpu().numpy()
        return self.postprocess(heatmaps, org_w, org_h)

    def _inference_onnx(self, img: np.ndarray) -> np.ndarray:
        # Prepare input data
        img_input, org_h, org_w = self.pre_img(img)

        # Feed to model
        ort_inputs = {self._ort_session.get_inputs()[0].name: img_input}
        heatmaps = self._ort_session.run(None, ort_inputs)[0]
        return self.postprocess(heatmaps, org_w, org_h)

    def _inference_trt(self, img: np.ndarray) -> np.ndarray:
        # Prepare input data
        img_input, org_h, org_w = self.pre_img(img)

        # Copy the data to appropriate memory
        np.copyto(self._inputs[0].host, img_input.ravel())
        heatmaps = engine_utils.do_inference(context=self._context,
                                             bindings=self._bindings,
                                             inputs=self._inputs,
                                             outputs=self._outputs,
                                             stream=self._stream)[0]

        # Reshape to output size
        heatmaps = heatmaps.reshape(1, 25, img_input.shape[2] // 4, img_input.shape[3] // 4)
        return self.postprocess(heatmaps, org_w, org_h)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VitInference(model_path, yolo_path, MODEL_SIZE, yolo_size=320, is_video=False, device = device)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-8-27 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

Loading /root/.cache/huggingface/hub/models--JunkyByte--easy_ViTPose/snapshots/2e599f9067ef175c7e270bafca586d1cf8d3f9df/yolov5/yolov5n.onnx for ONNX Runtime

[Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/Pillow-9.4.0.dist-info/METADATA'


Adding AutoShape... 


# Dataset
- model.inference가 torch.tensor가 아닌 ndarray type을 input으로 받아야 해서 dataloader 사용 X

In [ ]:
path =  '/content/gdrive/MyDrive/KUBIG/23_summer_CV/pose-estimation-project/원천데이터'

In [ ]:
import os
from glob import glob

In [ ]:
folders = os.listdir(path)

In [ ]:
images = []
for i in folders:
  folder_name = i.split('(')[0].replace(' ', '')
  for j in glob(path + f'/{i}/*'):
    images.append([folder_name, j])
images[:5]

[['앞서고안막기',
  '/content/gdrive/MyDrive/KUBIG/23_summer_CV/pose-estimation-project/원천데이터/앞서고 안막기/B-005-A036-F-B2006-S-20211126-01-01-E01.jpg'],
 ['앞서고안막기',
  '/content/gdrive/MyDrive/KUBIG/23_summer_CV/pose-estimation-project/원천데이터/앞서고 안막기/B-005-A036-F-B2006-S-20211126-01-07-E01.jpg'],
 ['앞서고안막기',
  '/content/gdrive/MyDrive/KUBIG/23_summer_CV/pose-estimation-project/원천데이터/앞서고 안막기/B-005-A035-M-B2006-S-20211109-01-01-E01.jpg'],
 ['앞서고안막기',
  '/content/gdrive/MyDrive/KUBIG/23_summer_CV/pose-estimation-project/원천데이터/앞서고 안막기/B-005-A035-M-B2006-S-20211109-01-07-E01.jpg'],
 ['앞서고안막기',
  '/content/gdrive/MyDrive/KUBIG/23_summer_CV/pose-estimation-project/원천데이터/앞서고 안막기/B-005-A037-M-C1999-S-20211104-01-01-E01.jpg']]

In [ ]:
import pandas as pd
data = pd.DataFrame(images)
data.columns = ['label','path']

In [ ]:
data.label.value_counts()

앞굽이하고지르기                      446
앞서고지르기                           290
앞굽이하고안막기                    162
뒷굽이하고손날거들어바깥막기     101
범서고바탕손안막기                  80
뒷굽이하고바깥막기                   80
앞굽이하고등주먹앞치기              80
앞굽이하고아래막기                    80
옆차기                                  74
주춤서고손날옆막기                  64
주춤서고팔꿈치표적치기               64
옆서고메주먹내려치기                   61
앞굽이하고헤쳐막기                    60
앞서고손날안치기                     60
앞서고아래막기                         60
앞서고등주먹바깥치기                 60
앞굽이하고팔꿈치거들어돌려치기     60
앞굽이하고팔꿈치표적치기            60
주춤서고옆지르기                       53
두발당성차기                          44
앞차기                                  42
범서고바깥막기                        42
돌려차기                       

In [ ]:
len(data)

3118

In [ ]:
label_dict = {i : idx for idx, i in enumerate(data.label.unique())}

In [ ]:
data['label'] = data['label'].map(label_dict)

In [ ]:
# import torch
# import cv2
# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# transform = A.Compose([
#     A.HorizontalFlip(p=0.5),
#     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#     # ToTensorV2(),
# ])

# class CustomImageDataset(torch.utils.data.Dataset):
#     def __init__(self, label, dir, tf):
#           self.img_labels = label
#           self.img_dir = dir
#           self.transform = tf

#     def __len__(self):
#         return len(self.img_labels)

#     def __getitem__(self, idx):
#         image = cv2.cvtColor(cv2.imread(self.img_dir.iloc[idx]), cv2.COLOR_BGR2RGB)
#         label = self.img_labels.iloc[idx]
#         transformed = self.transform(image=image)
#         image = transformed['image']
#         return image, label

In [ ]:
# TaekwonDataset = CustomImageDataset(data['label'], data['path'], transform)

In [ ]:
# TaekwonDataloader = torch.utils.data.DataLoader(TaekwonDataset, shuffle=True)

# From TaekwonDataset infer keypoint

In [ ]:
from tqdm.auto import tqdm
keypoints = []
labels = []

for i in tqdm(zip(data['label'], data['path'])):
  label = i[0]
  keypoint = model.inference(cv2.cvtColor(cv2.imread(i[1]), cv2.COLOR_BGR2RGB))
  keypoints.append(keypoint)
  labels.append(label)

0it [00:00, ?it/s]

In [ ]:
# np.save('/content/gdrive/MyDrive/kubig_pose/keypoints', np.array(keypoints))

In [ ]:
# np.save('/content/gdrive/MyDrive/kubig_pose/labels', np.array(labels))

In [ ]:
keypoints = np.load("/content/gdrive/MyDrive/KUBIG/23_summer_CV/pose-estimation-project/keypoints.npy", allow_pickle=True)
labels = np.load("/content/gdrive/MyDrive/KUBIG/23_summer_CV/pose-estimation-project/labels.npy", allow_pickle=True)

In [ ]:
keypoints[0]

{0: array([[     205.18,      966.68,     0.98418],
        [     194.08,       979.2,     0.96536],
        [     193.28,      956.04,     0.96592],
        [     198.34,      996.66,     0.97631],
        [      196.6,      940.43,      1.0015],
        [     261.83,      969.21,     0.96582],
        [     260.41,      1025.2,     0.95469],
        [     265.76,      912.55,     0.93422],
        [     307.68,      1042.9,     0.94538],
        [      325.4,      925.59,     0.84176],
        [     345.48,      1030.7,     0.98166],
        [      324.1,      955.86,     0.96419],
        [     414.93,      994.64,     0.84382],
        [     411.43,      932.25,     0.84641],
        [     413.36,      962.92,     0.86396],
        [     537.28,      991.11,     0.89198],
        [     521.97,      937.44,     0.91169],
        [     654.61,      987.88,     0.92108],
        [     598.51,      943.22,     0.95119],
        [     698.29,      978.86,     0.97126],
        [     690

In [ ]:
labels

array([ 0,  0,  0, ..., 46, 46, 46])

## 데이터 증강 by Flipping keypoints




이미지 size 확인

In [ ]:
import cv2

# 이미지 파일 경로
image_path = "/content/gdrive/MyDrive/KUBIG/23_summer_CV/pose-estimation-project/원천데이터/뒷굽이하고 손날바깥막기/B-010-A035-M-B2006-S-20211109-01-01-E01.jpg"  # 이미지 임의로 하나 가져오기

# 이미지 로드
image = cv2.imread(image_path)

# 이미지의 width 출력
image_width = image.shape[1]
image_height = image.shape[0]
print("Image width:", image_width)
print("Image height:", image_height)

Image width: 1920
Image height: 1080


###keypoint 좌우반전하기 (무시 ! 아래에서 진행)

In [ ]:
# import numpy as np

# def flip_keypoints(keypoints, image_width):
#     flipped_keypoints = []

#     for keypoint_dict in keypoints:
#         flipped_keypoint_dict = {}
#         for idx, coordinates in keypoint_dict[0].items():
#             flipped_x = image_width - coordinates[0]
#             flipped_keypoint_dict[idx] = [flipped_x, coordinates[1], coordinates[2]]
#         flipped_keypoints.append({0: flipped_keypoint_dict})

#     return flipped_keypoints

# # 원본 데이터셋에서 관절 좌표와 라벨을 가져오는 코드
# # (관절 좌표는 keypoints, 라벨은 labels로 가정)

# augmented_keypoints = []
# augmented_labels = []

# for original_keypoints, label in zip(keypoints, labels):
#     # 이미지의 폭을 얻어오거나 직접 입력해야 함
#     image_width = 1920

#     # 좌우반전된 관절 좌표 생성
#     flipped_keypoints = flip_keypoints(original_keypoints, image_width)

#     augmented_keypoints.append(original_keypoints)
#     augmented_keypoints.append(flipped_keypoints)

#     augmented_labels.append(label)
#     augmented_labels.append(label)

# # 증강된 데이터를 기존 데이터에 추가
# keypoints.extend(augmented_keypoints)
# labels.extend(augmented_labels)

TypeError: ignored

이걸로도 성능 개선이 안 되면 아래 중 하나로 진행해야할 듯
- bbox 내 좌표로 통일 (종우님 의견)
- 좌표간 각도 계산 (민규님 의견)

# From keypoint to classification

In [ ]:
def keypoint_flatten(keypoints, label):
  columns = []

  for i in range(25):
    columns.extend([f'y{i}', f'x{i}'])
  columns.append('label')
  ml_data = pd.DataFrame(columns = columns)


  for idx, keypoint in enumerate(keypoints):
    flattend_data = []

    try :
      for i in keypoint[0]:
        flattend_data.append(i[0])
        flattend_data.append(i[1])
      flattend_data.append(label[idx])
      ml_data.loc[idx] = flattend_data
    except :
      pass
  return ml_data

In [ ]:
df = keypoint_flatten(keypoints, labels)

In [ ]:
df

,y0,x0,y1,x1,y2,...,y23,x23,y24,x24,label
0,205.177368,966.683228,194.080811,979.200867,193.277130,...,619.279541,932.659607,601.986206,948.716980,0.0
1,211.253021,567.365601,205.044968,570.523926,204.725693,...,506.744324,644.375732,505.097046,678.189026,0.0
2,149.869995,943.557556,140.361084,953.411743,140.619995,...,520.015259,914.174866,504.355469,924.018311,0.0
3,179.628754,718.676331,173.180954,724.367920,172.443756,...,491.723633,779.317139,493.803284,818.889893,0.0
4,100.417236,921.562256,92.367981,929.884583,92.681030,...,427.179443,890.333801,413.665283,904.144775,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3113,196.493484,1314.469971,189.784485,1313.710449,189.944000,...,497.183472,1254.314331,473.488159,1233.499023,46.0
3114,177.668182,961.745056,167.950562,972.582397,167.565002,...,525.771240,941.714966,514.239136,960.305176,46.0
3115,176.036591,1284.121582,169.589294,1281.888916,170.812027,...,488.566284,1213.017944,467.897583,1183.031982,46.0
3116,194.740814,905.981506,183.517212,916.914124,184.433868,...,560.718750,888.581787,548.370850,916.819763,46.0


keypoint 데이터 자체에서 증강하려고 했으나 만들어진 df 에서 x 좌표만 image width 인 1920에서 뺀 값으로 대체해주는 것이 더 간단함.

# 데이터 증강 by Flipping Keypoints

In [ ]:
# 새로운 DataFrame 생성하여 추가할 데이터 저장
augmented_data = []

for idx, row in df.iterrows():
    augmented_row = {}
    for col_name, value in row.items():
        if col_name.startswith('x'):
            new_value = 1920 - value  # 1920에서 뺀 값으로 대체
            augmented_row[col_name] = new_value
        else:
            augmented_row[col_name] = value
    augmented_data.append(augmented_row)

# 새로운 데이터를 포함한 DataFrame 생성
augmented_df = pd.DataFrame(augmented_data)

# 원래 데이터와 확장된 데이터 결합
final_df = pd.concat([df, augmented_df], ignore_index=True)


In [ ]:
final_df

,y0,x0,y1,x1,y2,x2,y3,x3,y4,x4,...,x20,y21,x21,y22,x22,y23,x23,y24,x24,label
0,205.177368,966.683228,194.080811,979.200867,193.277130,956.036133,198.341705,996.658386,196.598053,940.432983,...,996.693298,663.879761,985.244324,625.850891,944.276978,619.279541,932.659607,601.986206,948.716980,0.0
1,211.253021,567.365601,205.044968,570.523926,204.725693,568.767456,208.802551,584.758667,206.749847,596.310120,...,572.351685,524.293701,603.514954,510.194702,641.990051,506.744324,644.375732,505.097046,678.189026,0.0
2,149.869995,943.557556,140.361084,953.411743,140.619995,933.401855,142.400513,968.180298,142.489136,917.887390,...,970.064636,552.726501,960.980225,524.739380,925.247986,520.015259,914.174866,504.355469,924.018311,0.0
3,179.628754,718.676331,173.180954,724.367920,172.443756,720.185669,177.456818,745.645386,177.200897,743.078064,...,703.259399,514.043701,743.569458,495.381897,776.937500,491.723633,779.317139,493.803284,818.889893,0.0
4,100.417236,921.562256,92.367981,929.884583,92.681030,913.307129,94.934174,941.867188,95.772644,900.709778,...,942.784180,454.414490,935.425110,431.305542,899.672852,427.179443,890.333801,413.665283,904.144775,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6127,196.493484,605.530029,189.784485,606.289551,189.944000,609.531616,193.228424,627.896362,193.197418,630.374268,...,581.988403,460.309906,613.047363,497.889282,655.788086,497.183472,665.685669,473.488159,686.500977,46.0
6128,177.668182,958.254944,167.950562,947.417603,167.565002,967.796021,166.750366,929.607666,166.284088,980.794434,...,934.357788,538.714966,943.477905,531.498291,971.658325,525.771240,978.285034,514.239136,959.694824,46.0
6129,176.036591,635.878418,169.589294,638.111084,170.812027,640.075073,174.048248,666.471558,178.006866,657.182861,...,622.676758,473.788757,657.855835,486.925537,697.216309,488.566284,706.982056,467.897583,736.968018,46.0
6130,194.740814,1014.018494,183.517212,1003.085876,184.433868,1026.253784,184.400299,988.079529,187.318787,1043.245239,...,987.669189,602.531982,1005.582092,567.274658,1024.159607,560.718750,1031.418213,548.370850,1003.180237,46.0


In [ ]:
final_df.to_csv('/content/gdrive/MyDrive/KUBIG/23_summer_CV/pose-estimation-project/final_df',  index=False)

# 1. RandomForest(ML) -> with tabular data

Flipping Augmentation으로 랜덤포레스트 성능 두 배 가까이 향상

In [ ]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(x, y, stratify = y, test_size = 0.15, random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [ ]:
model.fit(train_x, train_y)

RandomForestClassifier()

In [ ]:
model.score(valid_x, valid_y)

0.925

# 2. FNN (Feedforward Neural Network) with data scailing

- dropout 적용 : 46.74%
- batch normalization 적용 : 48.7%
- hidden_size 64 -> 128 : 48.91%


성능이 오히려 하락된 것
- learning rate scheduler
- L2 normalization


In [ ]:
# 데이터 스케일링
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
valid_x_scaled = scaler.transform(valid_x)

# 데이터를 PyTorch의 DataLoader로 변환
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_x_scaled, dtype=torch.float32),
                                              torch.tensor(train_y.values, dtype=torch.long))
valid_dataset = torch.utils.data.TensorDataset(torch.tensor(valid_x_scaled, dtype=torch.float32),
                                              torch.tensor(valid_y.values, dtype=torch.long))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 신경망 모델 정의
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_prob):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)  # Batch Normalization 추가
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)  # Dropout 추가
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)  # Batch Normalization 적용
        x = self.relu(x)
        x = self.dropout(x)  # Dropout 적용
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# 모델 초기화 및 손실 함수, 옵티마이저 설정
input_size = train_x_scaled.shape[1]
hidden_size = 128
output_size = len(np.unique(train_y))
dropout_prob = 0.15  # Dropout 확률 설정 (최적화 필요)
model = FNN(input_size, hidden_size, output_size, dropout_prob)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early Stopping 설정
best_valid_loss = float('inf')
patience = 3  # 몇 번의 epoch 동안 검증 손실이 감소하지 않으면 학습을 중단할지 설정

# 모델 학습
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    valid_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_valid_loss = valid_loss / len(valid_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Validation Loss: {avg_valid_loss:.4f} - "
          f"Validation Accuracy: {(100 * correct / total):.2f}%")

    # Early Stopping 검증
    if avg_valid_loss < best_valid_loss:
        best_valid_loss = avg_valid_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

Epoch [1/100] - Validation Loss: 3.6892 - Validation Accuracy: 20.00%
Epoch [2/100] - Validation Loss: 3.6727 - Validation Accuracy: 21.74%
Epoch [3/100] - Validation Loss: 3.6538 - Validation Accuracy: 23.91%
Epoch [4/100] - Validation Loss: 3.6285 - Validation Accuracy: 27.17%
Epoch [5/100] - Validation Loss: 3.6270 - Validation Accuracy: 26.63%
Epoch [6/100] - Validation Loss: 3.6150 - Validation Accuracy: 27.72%
Epoch [7/100] - Validation Loss: 3.5962 - Validation Accuracy: 30.00%
Epoch [8/100] - Validation Loss: 3.5842 - Validation Accuracy: 32.39%
Epoch [9/100] - Validation Loss: 3.5736 - Validation Accuracy: 32.17%
Epoch [10/100] - Validation Loss: 3.5481 - Validation Accuracy: 35.54%
Epoch [11/100] - Validation Loss: 3.5314 - Validation Accuracy: 37.39%
Epoch [12/100] - Validation Loss: 3.5230 - Validation Accuracy: 38.15%
Epoch [13/100] - Validation Loss: 3.5139 - Validation Accuracy: 39.13%
Epoch [14/100] - Validation Loss: 3.5032 - Validation Accuracy: 39.57%
Epoch [15/100] 

standardization 추가

In [ ]:
standardized_df = final_df.copy()

def standardization(value, mean, std):
  return (value - mean) / std

for idx, row in standardized_df.iterrows():
    x_coord = []
    y_coord = []
    for col_name, value in row.items():
        if col_name.startswith('x'):
          x_coord.append(value)
        elif col_name.startswith('y'):
          y_coord.append(value)

    x_mean = np.array(x_coord).mean()
    x_std = np.array(x_coord).std()
    y_mean = np.array(y_coord).mean()
    y_std = np.array(y_coord).std()

    for col_name, value in row.items():
        if col_name.startswith('x'):
          standardized_df.loc[idx, col_name] = standardization(value, x_mean, x_std)
        elif col_name.startswith('y'):
          standardized_df.loc[idx, col_name] = standardization(value, y_mean, y_std)

standardized_df[:5]

,y0,x0,y1,x1,y2,x2,y3,x3,y4,x4,...,x20,y21,x21,y22,x22,y23,x23,y24,x24,label
0,-1.211710,-0.084188,-1.273957,0.292913,-1.278465,-0.404937,-1.250055,0.818830,-1.259836,-0.874990,...,0.819882,1.361415,0.474975,1.148089,-0.759188,1.111227,-1.109168,1.014218,-0.625431,0.0
1,-1.229617,-0.876085,-1.280594,-0.789579,-1.283216,-0.837689,-1.249739,-0.399690,-1.266595,-0.083295,...,-0.739517,1.340933,0.114045,1.225158,1.167877,1.196825,1.233221,1.183299,2.159367,0.0
2,-1.223080,-0.091482,-1.283774,0.267598,-1.282121,-0.461548,-1.270756,0.805753,-1.270191,-1.026884,...,0.874417,1.348272,0.543388,1.169636,-0.758669,1.139483,-1.162166,1.039530,-0.803478,0.0
3,-1.244221,-0.738662,-1.294174,-0.586785,-1.299885,-0.698386,-1.261048,-0.019007,-1.263030,-0.087514,...,-1.150055,1.346590,-0.074402,1.202012,0.816008,1.173670,0.879507,1.189782,1.935486,0.0
4,-1.229815,-0.046309,-1.288162,0.285960,-1.285892,-0.375895,-1.269560,0.764366,-1.263482,-0.878845,...,0.800977,1.336187,0.507166,1.168678,-0.920245,1.138770,-1.293106,1.040810,-0.741703,0.0


# 3. VGGnet-16 시도

In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models import vgg16
from sklearn.metrics import accuracy_score

저장해둔 증강 데이터를 불러오기

In [ ]:
final_df = pd.read_csv('/content/gdrive/MyDrive/KUBIG/23_summer_CV/pose-estimation-project/final_df')

In [ ]:
x = final_df.drop(columns = 'label')
y = final_df['label']

### 방법 1) 관절좌표 데이터를 이미지로 변환하고 모델 돌리기

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

num_keypoints = 25

# 관절 좌표를 이미지로 변환 (픽셀 좌표로 매핑하여 이미지 생성)
image_size = (1080, 1920)  # 이미지 크기 설정

# 관절 좌표 데이터로부터 이미지 생성
num_samples = len(final_df)
images = np.zeros((num_samples, 1, image_size[0], image_size[1]), dtype=np.float32)

for i in range(num_samples):
    for j in range(25):
        y = int(final_df[f'x{j}'][i] * image_size[0])
        y = max(min(y, image_size[0] - 1), 0)  # 범위 내에서 조정
        x = int(final_df[f'y{j}'][i] * image_size[1])
        x = max(min(x, image_size[1] - 1), 0)  # 범위 내에서 조정
        images[i, 0, y, x] = 1  # 관절을 점으로 표시


In [ ]:
images[0]

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)

In [ ]:
images_tensor = torch.tensor(images, dtype=torch.float32)  # 텐서로 변환

In [ ]:
train_ratio = 0.85  # 85%를 학습용으로 사용
# 데이터를 학습용과 검증용으로 나누기
train_indices, valid_indices = train_test_split(range(len(final_df)), train_size=train_ratio, random_state=42, stratify=final_df['label'])

In [ ]:
import torch.nn.functional as F
from torchvision.models import vgg16

class ModifiedVGG(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedVGG, self).__init__()
        self.vgg_model = vgg16(pretrained=True)
        self.vgg_model.classifier[0] = nn.Linear(128, 4096)  # 입력 크기 조정
        self.vgg_model.classifier[6] = nn.Linear(4096, num_classes)  # 출력 크기 조정

    def forward(self, x):
        x = x.view(x.size(0), -1)  # 1D로 변환
        x = self.vgg_model(x)
        return x

# 모델 초기화
num_classes = 47
model = ModifiedVGG(num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(model)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ModifiedVGG(
  (vgg_model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
    

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
# 이미지 전처리를 위한 변환 함수 정의
transform = transforms.Compose([
    transforms.ToPILImage(),  # 텐서를 이미지로 변환
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),  # 이미지를 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 정규화
])

아래 코드에서 RAM 문제로 세션이 다운됨.

In [ ]:
# CustomDataset을 사용하여 DataLoader 생성
train_dataset = CustomDataset(images_tensor[train_indices], final_df['label'].values[train_indices], transform=transform)
valid_dataset = CustomDataset(images_tensor[valid_indices], final_df['label'].values[valid_indices], transform=transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

### 방법 2) 데이터를 2d array 로 변환

In [ ]:
def from_df_to_cnn(df):
  data = df.drop(columns = 'label')
  labels = df['label']

  data_list = []

  for idx, rows in data.iterrows():
    data_list.append(np.array(rows).reshape(-1,2))

  return np.array(data_list), np.array(labels)

In [ ]:
x, y = from_df_to_cnn(standardized_df)

In [ ]:
from sklearn.model_selection import train_test_split
train_x, valid_x, train_y, valid_y = train_test_split(x, y, stratify = y, test_size = 0.15, random_state = 42)

In [ ]:
class CustomKeypointsDataset(torch.utils.data.Dataset):
    def __init__(self, keypoints, labels):
        self.keypoints = keypoints
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        keypoint = torch.tensor(self.keypoints[idx].T, dtype = torch.float)
        label = torch.tensor(self.labels[idx], dtype = torch.float)
        return keypoint, label

In [ ]:
SkeletonTrainDataset = CustomKeypointsDataset(train_x, train_y)
SkeletonValidDataset = CustomKeypointsDataset(valid_x, valid_y)

In [ ]:
TrainDataloader = torch.utils.data.DataLoader(SkeletonTrainDataset, batch_size = 128, shuffle = True)
ValidDataloader = torch.utils.data.DataLoader(SkeletonValidDataset, batch_size = 128,shuffle = False)

In [ ]:
from torchvision.models import vgg16
import torch.nn as nn

class ModifiedVGG(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(ModifiedVGG, self).__init__()
        vgg_model = vgg16(pretrained=True)

        # 첫 번째 컨볼루션 레이어의 in_channels 수정
        vgg_model.features[0] = nn.Conv2d(in_channels, 64, kernel_size=3, padding=1)

        # MaxPooling 레이어의 사이즈 변경
        vgg_model.features[4] = nn.MaxPool2d(kernel_size=2, stride=1)  # 이 부분을 변경

        # 마지막 fully connected layer 변경
        vgg_model.classifier[6] = nn.Linear(4096, num_classes)

        self.features = vgg_model.features
        self.classifier = vgg_model.classifier[:-1]  # 마지막 레이어를 제외한 레이어들 사용

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# 모델 초기화
in_channels = 128
num_classes = 47  # 예시로 2개 클래스로 설정
model = ModifiedVGG(in_channels, num_classes)
# 모델을 원하는 디바이스로 옮기기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(model)

# 손실 함수와 최적화기 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg_model.parameters(), lr=0.001)

ModifiedVGG(
  (features): Sequential(
    (0): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, 

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
  train_loss = []
  train_acc = []
  valid_loss = []
  valid_acc = []

  for skeleton, cls in iter(TrainDataloader):
    optimizer.zero_grad()

    skeleton, cls = skeleton.to(device), cls.to(device).long()

    output = model(skeleton)
    loss = criterion(output, cls)
    train_loss.append(loss.item())
    train_acc.append(sum(torch.max(output, dim=1)[1] == cls) / skeleton.shape[0])
    loss.backward()
    optimizer.step()

  with torch.no_grad():
    for skeleton, cls in iter(ValidDataloader):
      skeleton, cls = skeleton.to(device), cls.to(device).long()
      output = model(skeleton)
      loss = criterion(output, cls)
      valid_loss.append(loss.item())
      valid_acc.append(sum(torch.max(output, dim=1)[1] == cls) / skeleton.shape[0])

  print(f'epoch {epoch+1} -- train_loss : {sum(train_loss[-len(TrainDataloader):]) / len(TrainDataloader):.5f} \
  train_acc : {sum(train_acc[-len(TrainDataloader):]) / len(TrainDataloader):.5f}\
   valid_loss : {sum(valid_loss[-len(ValidDataloader):]) / len(ValidDataloader):.5f} \
  valid_acc : {sum(valid_acc[-len(ValidDataloader):]) / len(ValidDataloader):.5f}')

RuntimeError: ignored

입출력 크기가 안 맞는 문제 발생